In [2]:
import torch
import math
import cmath

In [60]:
n_qubits = 3

In [67]:
random_tensor = torch.rand(2**n_qubits, dtype=torch.cfloat)
sum_elements = torch.sum(random_tensor)
initial_state = random_tensor/sum_elements

In [68]:
initial_state

tensor([0.0802+0.0637j, 0.1016-0.0212j, 0.1770-0.0939j, 0.0836-0.0278j,
        0.0878+0.1023j, 0.2613+0.0713j, 0.0465-0.0204j, 0.1620-0.0740j])

In [69]:
contracted_tensor = initial_state.view(*(2,)*n_qubits)
contracted_tensor

tensor([[[0.0802+0.0637j, 0.1016-0.0212j],
         [0.1770-0.0939j, 0.0836-0.0278j]],

        [[0.0878+0.1023j, 0.2613+0.0713j],
         [0.0465-0.0204j, 0.1620-0.0740j]]])

In [55]:


initial_state = torch.tensor([0.55, 0.2, 0.25, 0.0],dtype=torch.complex64)
identity_tensor = torch.eye(2, dtype=torch.complex64)

H = [[(1/math.sqrt(2)), (1/math.sqrt(2))], [(1/math.sqrt(2)), -(1/math.sqrt(2))]]
theta_value = 2*math.pi

hadamard_gate = torch.tensor(H, dtype=torch.complex64)

rx_gate = torch.tensor([[math.cos(theta_value/2), -1j*math.sin(theta_value/2)],
                        [-1j*math.sin(theta_value/2), math.cos(theta_value/2)]], dtype=torch.complex64)

ry_gate = torch.tensor([[math.cos(theta_value/2), -math.sin(theta_value/2)], [math.sin(theta_value/2), math.cos(theta_value/2)]], dtype=torch.complex64)

rz_gate = torch.tensor([[-cmath.exp(-1.0j*theta_value/2), 0.0], [0.0, cmath.exp(-1.0j*theta_value/2)]], dtype=torch.complex64)
 
pauli_x_gate = torch.tensor([[0.0, 1.0],[1.0 ,0.0]], dtype=torch.complex64)

pauli_y_gate = torch.tensor([[0.0, -1.0j], [1.0j, 0.0]],dtype=torch.complex64)
pauli_z_gate = torch.tensor([[1.0, 0.0], [0.0,-1.0]], dtype=torch.complex64)

phase_gate = torch.tensor([[1.0, 0.0], [0.0, 1.0j]], dtype=torch.complex64)
t_gate = torch.tensor([[1.0, 0.0], [0.0, cmath.exp(math.pi*1.0j/4)]], dtype=torch.complex64)

In [4]:
gate_operation = {
    'pauli_x': pauli_x_gate,
    'pauli_y': pauli_y_gate,
    'pauli_z': pauli_z_gate,
    'rx_gate': rx_gate,
    'ry_gate': ry_gate,
    'rz_gate': rz_gate,
    'phase': phase_gate,
    't': t_gate,
    'hadamard': hadamard_gate
}

In [45]:
for i in range(n_qubits - 1):
    U = torch.kron(hadamard_gate, identity_tensor)
state = U @ initial_state
print(state)

RuntimeError: size mismatch, got input (6), mat (6x6), vec (8)

In [76]:
next_state = torch.tensordot(pauli_x_gate, contracted_tensor, dims=([1], [2]))
next_state = torch.tensordot(rx_gate, next_state, dims=([1],[2]))
next_state

tensor([[[-0.1016+0.0212j, -0.2613-0.0713j],
         [-0.0802-0.0637j, -0.0878-0.1023j]],

        [[-0.0836+0.0278j, -0.1620+0.0740j],
         [-0.1770+0.0939j, -0.0465+0.0204j]]])

In [50]:
def run_circuit(tensor_expression: list, angle: float, target_qubit) -> torch.Tensor:
    next_state = angle
    for gate_name in tensor_expression:
        gate = gate_operation[gate_name]
        next_state = torch.tensordot(gate, next_state, dims=([1],[target_qubit]))
        print(next_state)
    
    return next_state

In [51]:
new_state = torch.empty((0,), dtype=torch.complex64)
target_qubit = 1
for angle in initial_state:
    new_angle = run_circuit(['rx_gate', 'pauli_x', 'hadamard'], angle, target_qubit)
    new_state = torch.cat((new_state, new_angle), dim=0)
    target_qubit+=1

IndexError: Dimension specified as 1 but tensor has no dimensions

In [25]:
print(new_state)

tensor([[-0.0000+0.0000e+00j,  0.0000+0.0000e+00j, -0.3889+0.0000e+00j,
          0.0000-4.7628e-17j, -0.0000+0.0000e+00j,  0.0000+0.0000e+00j,
         -0.3889+0.0000e+00j,  0.0000-4.7628e-17j],
        [ 0.0000+0.0000e+00j, -0.0000+0.0000e+00j,  0.0000-4.7628e-17j,
         -0.3889+0.0000e+00j,  0.0000+0.0000e+00j, -0.0000+0.0000e+00j,
          0.0000-4.7628e-17j, -0.3889+0.0000e+00j],
        [-0.3889+0.0000e+00j,  0.0000-4.7628e-17j, -0.0000+0.0000e+00j,
          0.0000+0.0000e+00j, -0.3889+0.0000e+00j,  0.0000-4.7628e-17j,
         -0.0000+0.0000e+00j,  0.0000+0.0000e+00j],
        [ 0.0000-4.7628e-17j, -0.3889+0.0000e+00j,  0.0000+0.0000e+00j,
         -0.0000+0.0000e+00j,  0.0000-4.7628e-17j, -0.3889+0.0000e+00j,
          0.0000+0.0000e+00j, -0.0000+0.0000e+00j],
        [-0.0000+0.0000e+00j,  0.0000+0.0000e+00j, -0.3889+0.0000e+00j,
          0.0000-4.7628e-17j,  0.0000-0.0000e+00j, -0.0000+0.0000e+00j,
          0.3889-0.0000e+00j,  0.0000+4.7628e-17j],
        [ 0.0000+0.0

In [56]:
no_qubits = 2
initial_state_vector = torch.eye(no_qubits, dtype=torch.complex64)
quantum_processing = run_circuit(['hadamard', 'rx_gate', 'pauli_x', 't'], initial_state_vector)
quantum_processing

tensor([[-0.7071-8.6596e-17j,  0.7071-8.6596e-17j],
        [-0.5000-5.0000e-01j, -0.5000-5.0000e-01j]])